In [ ]:
import dspy
import re
from dspy.primitives.python_interpreter import PythonInterpreter, CodePrompt

# Configure a local model with Ollama
# lm = dspy.LM('ollama_chat/tulu3', api_base='https://xxxx-xxx-xx-xxx-xx.ngrok-free.app', api_key='') #api_key should be empty
# lm = dspy.LM('ollama_chat/tulu3', api_base='http://localhost:11434', api_key='') #api_key should be empty

# Read the API key from a file
with open('/home/c/carlosm/.openaikey.txt', 'r') as file:
    api_key = file.read().strip()

# Configure the language model with OpenAI GPT-4o
lm = dspy.LM('openai/gpt-4o', api_key=api_key)
dspy.settings.configure(lm=lm)


class ProgramOfThought(dspy.Module):
    """Class to generate and execute Python code iteratively to solve questions."""

    def __init__(self, signature, max_iters=3):
        super().__init__()
        self.signature = signature
        self.max_iters = max_iters
        # Allow importing only numpy, astropy, and sympy
        self.interpreter = PythonInterpreter(
            action_space={"print": print},
            import_white_list=["numpy", "astropy", "sympy"]
        )

    def forward(self, question):
        """Generates and executes Python code for a given question up to a maximum number of iterations."""

        for iteration in range(self.max_iters):
            # Generate Python code for the question
            generated_code = self._generate_code(question)
            print(f"\nIteration {iteration + 1}:")
            print("Generated Code:\n", generated_code)

            # Attempt to execute the code
            try:
                result, _ = CodePrompt(generated_code).execute(self.interpreter)
                print("Execution Result:\n", result)
                return {"answer": result}
            except Exception as e:
                print("Execution Error:\n", e)

        # Return None if all iterations fail
        print("\nFailed to generate valid code after several iterations.")
        return {"answer": None}

    def _generate_code(self, question):
        """Generates Python code using the model for the given question."""

        generate_code_signature = dspy.Signature("question -> generated_code")
        predict = dspy.Predict(generate_code_signature)

        prompt = f"""
You are an AI that writes Python code to solve problems. 
Generate Python code to solve the following problem. Do not include any explanations, Markdown formatting, or comments. 
Only output the executable Python code.

Question: {question}

Code:
"""
        # Use the model to generate the code
        completion = predict(question=prompt)
        generated_code = completion.generated_code

        # Clean code block tags like ```python and ```
        cleaned_code = re.sub(r"```[^\n]*\n", "", generated_code)
        cleaned_code = re.sub(r"```", "", cleaned_code)

        return cleaned_code


# Create an instance of ProgramOfThought with a simple signature
generate_answer_signature = dspy.Signature("question -> answer")
pot = ProgramOfThought(signature=generate_answer_signature)

# Example usage
question = "Chris saw 2 viscachas at lunch time, then 3 at dinner. How many viscachas did Chris see in the day?"
result = pot(question=question)

# Print the final answer
print("\nQuestion:", question)
print("Final Answer:", result['answer'])


In [ ]:
# Another example
question = (
    "An object in free fall travels a distance y(t) = 5t^2 meters as a function of time t in seconds. "
    "What is the object's acceleration and velocity at any instant t= 2.5?"
)
result = pot(question=question)

# Print the final answer
print("\nQuestion:", question)
print("Final Answer:", result['answer'])

In [ ]:
type(result)

In [ ]:
# Another example
question = "Using the astropy library, convert 1.5 parsecs to light-years."
result = pot(question=question)

# Print the final answer
print("\nQuestion:", question)
print("Final Answer:", result['answer'])

In [ ]:
# Another example
question = "A celestial object has equatorial coordinates of right ascension 10 hours and declination +20 degrees. Convert these coordinates to galactic coordinates using astropy."
result = pot(question=question)

# Print the final answer
print("\nQuestion:", question)
print("Final Answer:", result['answer'])

In [ ]:
# Another example
question = "A celestial object moves at a speed of 30,000 km/s. Calculate the relativistic redshift of this object using astropy."
result = pot(question=question)

# Print the final answer
print("\nQuestion:", question)
print("Final Answer:", result['answer'])

## Memory Implementation

In [ ]:
import dspy
import re
from dspy.primitives.python_interpreter import PythonInterpreter, CodePrompt
from pydantic import BaseModel

# Configure a local model with Ollama
# lm = dspy.LM('ollama_chat/tulu3', api_base='https://xxxx-xxx-xx-xxx-xx.ngrok-free.app', api_key='') #api_key should be empty
# lm = dspy.LM('ollama_chat/tulu3', api_base='http://localhost:11434', api_key='') #api_key should be empty

# Read the API key from a file
with open('/home/c/carlosm/.openaikey.txt', 'r') as file:
    api_key = file.read().strip()

# Configure the language model with OpenAI GPT-4o
lm = dspy.LM('openai/gpt-4o', api_key=api_key)
dspy.settings.configure(lm=lm)

# =====================
# Conversation Memory Classes
# =====================

class UserInteraction(BaseModel):
    message: str = None
    response: str = None

    def serialize_by_role(self):
        return [
            {
                "role": "user",
                "content": f"{self.message}"
            },
            {
                "role": "assistant",
                "content": f"{self.response}"
            }

        ]

class ConversationContext(BaseModel):
    window_size: int = 3
    content: list[UserInteraction] = []

    @staticmethod
    def _format_interaction(interaction):
        return f"User: {interaction.message}\n\nAssistant: {interaction.response}\n\n"

    def render(self):
        formatted = [
            self._format_interaction(interaction)
            for interaction in self.content
        ]
        return "".join(formatted)

    def update(self, interaction: UserInteraction):
        # Keep only the last `window_size` interactions
        self.content = self.content[-self.window_size:] + [interaction]

    def __str__(self):
        return self.render()


# =====================
# Signature and ChainOfThought for Responding with Context
# =====================

class ResponseWithContext(dspy.Signature):
    """Respond to the user's message given some conversation context"""
    context = dspy.InputField(desc="The context of the conversation")
    message = dspy.InputField(desc="The user's message")
    response = dspy.OutputField(desc="A useful response to the user's message within the context of the conversation")

respond_cot = dspy.ChainOfThought(ResponseWithContext)


# =====================
# ProgramOfThought Class (as before, but now including context)
# =====================

class ProgramOfThought(dspy.Module):
    """Class to generate and execute Python code iteratively to solve questions, now with context consideration."""

    def __init__(self, signature, max_iters=3):
        super().__init__()
        self.signature = signature
        self.max_iters = max_iters
        # Allow importing only numpy, astropy, and sympy
        self.interpreter = PythonInterpreter(
            action_space={"print": print},
            import_white_list=["numpy", "astropy", "sympy", "matplotlib", "mpmath"],
        )

    def forward(self, question, variables=None, context_str=""):
        """Generates and executes Python code for a given question up to a maximum number of iterations, using provided context for better answers."""
        for iteration in range(self.max_iters):
            # Generate Python code for the question, including context
            generated_code = self._generate_code(question, context_str)
            print(f"\nIteration {iteration + 1}:")
            print("Generated Code:\n", generated_code)

            # Attempt to execute the code with the provided variables
            try:
                result, _ = CodePrompt(generated_code).execute(self.interpreter, user_variable=variables)
                print("Execution Result:\n", result)
                return {"answer": result}
            except Exception as e:
                print("Execution Error:\n", e)

        # Return None if all iterations fail
        print("\nFailed to generate valid code after several iterations.")
        return {"answer": None}

    def _generate_code(self, question, context_str=""):
        """Generates Python code using the model for the given question, with additional conversation context."""
        generate_code_signature = dspy.Signature("question -> generated_code")
        predict = dspy.Predict(generate_code_signature)

        # We incorporate the conversation context into the prompt:
        # The model can use the previous Q&A to provide a better solution.
        prompt = f"""
You are an AI that writes Python code to solve problems. 
Below is the conversation context so far:

{context_str}

Now, generate Python code to solve the following problem. Do not include any explanations, Markdown formatting, or comments. 
Only output the executable Python code.

Question: {question}

Code:
"""
        # Use the model to generate the code
        completion = predict(question=prompt)
        generated_code = completion.generated_code

        # Clean code block tags like ```python and ```
        cleaned_code = re.sub(r"```[^\n]*\n", "", generated_code)
        cleaned_code = re.sub(r"```", "", cleaned_code)

        return cleaned_code


# =====================
# Example main loop integrating memory
# =====================

if __name__ == '__main__':

    # Create conversation context with a window size
    context = ConversationContext(window_size=5)

    # Create an instance of ProgramOfThought with a simple signature
    generate_answer_signature = dspy.Signature("question -> answer")
    pot = ProgramOfThought(signature=generate_answer_signature)

    while True:
        try:
            user_message = input(">>> ")

            if user_message == ':exit':
                break
            elif user_message == ':history':
                print(context.render())
                continue

            # Use respond_cot to get an assistant response based on the conversation history
            # This response simulates a "chain of thought" for a normal conversational turn
            assistant_response = respond_cot(
                context=context.render(),
                message=user_message
            ).response

            # Print the assistant response
            print(f'\n<<< {assistant_response}\n')

            # Update the conversation context
            interaction = UserInteraction(message=user_message, response=assistant_response)
            context.update(interaction)

            # Now, if the user message is a problem that requires code generation,
            # we can call pot using the current conversation context.
            # For example, if the user asks a physics question:
            if "calculate" in user_message.lower() or "solve" in user_message.lower():
                variables = {"gravity": 9.81}  # Example variable
                result = pot(question=user_message, variables=variables, context_str=context.render())
                print("ProgramOfThought Answer:", result['answer'])

        except KeyboardInterrupt:
            break

    print('\nBye!')

## RAG Implementation

In [ ]:
import dspy
import re
import pandas as pd
from dspy.primitives.python_interpreter import PythonInterpreter, CodePrompt
from pydantic import BaseModel

# Configure a local model with Ollama
# lm = dspy.LM('ollama_chat/tulu3', api_base='https://xxxx-xxx-xx-xxx-xx.ngrok-free.app', api_key='') #api_key should be empty
# lm = dspy.LM('ollama_chat/tulu3', api_base='http://localhost:11434', api_key='') #api_key should be empty

# Read the API key from a file
with open('/home/c/carlosm/.openaikey.txt', 'r') as file:
    api_key = file.read().strip()

# Configure the language model with OpenAI GPT-4o
lm = dspy.LM('openai/gpt-4o', api_key=api_key)
dspy.settings.configure(lm=lm)

LOG = logging.getLogger(__name__)
# =====================
# Conversation Memory Classes
# =====================

class UserInteraction(BaseModel):
    message: str = None
    response: str = None

    def serialize_by_role(self):
        return [
            {
                "role": "user",
                "content": f"{self.message}"
            },
            {
                "role": "assistant",
                "content": f"{self.response}"
            }
        ]


class ConversationContext(BaseModel):
    window_size: int = 3
    content: list[UserInteraction] = []

    @staticmethod
    def _format_interaction(interaction):
        return f"User: {interaction.message}\n\nAssistant: {interaction.response}\n\n"

    def render(self):
        formatted = [
            self._format_interaction(interaction)
            for interaction in self.content
        ]
        return "".join(formatted)

    def update(self, interaction: UserInteraction):
        self.content = self.content[-self.window_size:] + [interaction]

    def __str__(self):
        return self.render()


# =====================
# Signature and ChainOfThought for Responding with Context
# =====================

class ResponseWithContext(dspy.Signature):
    """Respond to the user's message given some conversation context."""
    context = dspy.InputField(desc="The context of the conversation")
    message = dspy.InputField(desc="The user's message")
    rag_context = dspy.InputField(desc="Relevant data from observing metadata")
    response = dspy.OutputField(desc="A useful response to the user's message within the context of the conversation")

respond_cot = dspy.ChainOfThought(ResponseWithContext)


# =====================
# Retrieval Function (RAG)
# =====================
def getObservingData(dayObs=None):
    """Get the observing metadata for the current or a past day.

    Get the observing metadata for the current or a past day. The metadata
    is the contents of the table on RubinTV. If a day is not specified, the
    current day is used. The metadata is returned as a pandas dataframe.

    Parameters
    ----------
    dayObs : `int`, optional
        The day for which to get the observing metadata. If not specified,
        the current day is used.

    Returns
    -------
    observingData: `pandas.DataFrame`
        The observing metadata for the specified day.
    """
    currentDayObs = 20240904
    if dayObs is None:
        dayObs = currentDayObs
    isCurrent = dayObs == currentDayObs

    site =  "local" #getSite()

    filename = None
    if site == "local":
        filename = f"/home/c/carlosm/lsst/summit_extras/python/lsst/summit/extras/dayObs_20240904.json"
    elif site in ["rubin-devl"]:
        LOG.warning(
            f"Observing metadata at {site} is currently copied by hand by Merlin and will"
            " not be updated in realtime"
        )
        filename = f"/sdf/home/m/mfl/u/rubinTvDataProducts/sidecar_metadata/dayObs_{dayObs}.json"
    elif site in ["staff-rsp"]:
        LOG.warning(
            f"Observing metadata at {site} is currently copied by hand by Merlin and will"
            " not be updated in realtime"
        )
        filename = f"/home/m/mfl/u/rubinTvDataProducts/sidecar_metadata/dayObs_{dayObs}.json"
    else:
        raise RuntimeError(f"Observing metadata not available for site {site}")

    # check the file exists, and raise if not
    if not os.path.exists(filename):
        LOG.warning(
            f"Observing metadata file for {'current' if isCurrent else ''} dayObs "
            f"{dayObs} does not exist at {filename}."
        )
        return pd.DataFrame()

    table = pd.read_json(filename).T
    table = table.sort_index()

    # remove all the columns which start with a leading underscore, as these
    # are used by the backend to signal how specific cells should be colored
    # on RubinTV, and for nothing else.
    table = table.drop([col for col in table.columns if col.startswith("_")], axis=1)

    return table

def retrieve_observing_metadata(query, dayObs=None):
    """Retrieve relevant observing data based on the query."""
    df = getObservingData(dayObs=dayObs)
    if df.empty:
        return "No observing data available."
    
    # Filter or search the dataframe based on the query
    result = df[df.apply(lambda row: query.lower() in row.to_string().lower(), axis=1)]
    return result.to_string(index=False) if not result.empty else "No relevant data found."


# =====================
# ProgramOfThought Class (Updated for RAG)
# =====================

class ProgramOfThought(dspy.Module):
    """Class to generate and execute Python code iteratively to solve questions, now with context consideration."""

    def __init__(self, signature, max_iters=3):
        super().__init__()
        self.signature = signature
        self.max_iters = max_iters
        self.interpreter = PythonInterpreter(
            action_space={"print": print},
            import_white_list=["numpy", "astropy", "sympy", "matplotlib"],
        )

    def forward(self, question, variables=None, context_str="", rag_context=""):
        """Generates and executes Python code for a given question."""
        for iteration in range(self.max_iters):
            generated_code = self._generate_code(question, context_str, rag_context)
            print(f"\nIteration {iteration + 1}:")
            print("Generated Code:\n", generated_code)

            try:
                result, _ = CodePrompt(generated_code).execute(self.interpreter, user_variable=variables)
                print("Execution Result:\n", result)
                return {"answer": result}
            except Exception as e:
                print("Execution Error:\n", e)

        print("\nFailed to generate valid code after several iterations.")
        return {"answer": None}

    def _generate_code(self, question, context_str="", rag_context=""):
        """Generates Python code using the model with RAG data."""
        generate_code_signature = dspy.Signature("question -> generated_code")
        predict = dspy.Predict(generate_code_signature)

        prompt = f"""
You are an AI that writes Python code to solve problems. 
Below is the conversation context and relevant observing metadata:

Context:
{context_str}

Observing Metadata:
{rag_context}

Now, generate Python code to solve the following problem. Do not include any explanations, Markdown formatting, or comments. 
Only output the executable Python code.

Question: {question}

Code:
"""
        completion = predict(question=prompt)
        generated_code = completion.generated_code

        cleaned_code = re.sub(r"```[^\n]*\n", "", generated_code)
        cleaned_code = re.sub(r"```", "", cleaned_code)

        return cleaned_code


# =====================
# Main Loop with RAG Integration
# =====================

if __name__ == '__main__':

    context = ConversationContext(window_size=5)
    generate_answer_signature = dspy.Signature("question -> answer")
    pot = ProgramOfThought(signature=generate_answer_signature)

    while True:
        try:
            user_message = input(">>> ")

            if user_message == ':exit':
                break
            elif user_message == ':history':
                print(context.render())
                continue

            rag_context = retrieve_observing_metadata(user_message)
            assistant_response = respond_cot(
                context=context.render(),
                message=user_message,
                rag_context=rag_context
            ).response

            print(f'\n<<< {assistant_response}\n')

            interaction = UserInteraction(message=user_message, response=assistant_response)
            context.update(interaction)

            if "calculate" in user_message.lower() or "solve" in user_message.lower():
                variables = {"gravity": 9.81}
                result = pot(question=user_message, variables=variables, context_str=context.render(), rag_context=rag_context)
                print("ProgramOfThought Answer:", result['answer'])

        except KeyboardInterrupt:
            break

    print('\nBye!')

In [ ]:
import dspy
import re
import os
import logging
import pandas as pd
from dspy.primitives.python_interpreter import PythonInterpreter, CodePrompt
from pydantic import BaseModel

# =====================
# Logging and LM Configuration
# =====================
LOG = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


# Configure a local model with Ollama
# lm = dspy.LM('ollama_chat/tulu3', api_base='https://xxxx-xxx-xx-xxx-xx.ngrok-free.app', api_key='') #api_key should be empty
# lm = dspy.LM('ollama_chat/tulu3', api_base='http://localhost:11434', api_key='') #api_key should be empty

# Read the API key from a file
with open('/home/c/carlosm/.openaikey.txt', 'r') as file:
    api_key = file.read().strip()

# Configure the language model with OpenAI GPT-4o
lm = dspy.LM('openai/gpt-4o', api_key=api_key)
dspy.settings.configure(lm=lm)

# =====================
# Conversation Memory Classes
# =====================

class UserInteraction(BaseModel):
    message: str = None
    response: str = None

    def serialize_by_role(self):
        return [
            {
                "role": "user",
                "content": f"{self.message}"
            },
            {
                "role": "assistant",
                "content": f"{self.response}"
            }
        ]


class ConversationContext(BaseModel):
    window_size: int = 3
    content: list[UserInteraction] = []

    @staticmethod
    def _format_interaction(interaction):
        return f"User: {interaction.message}\n\nAssistant: {interaction.response}\n\n"

    def render(self):
        formatted = [
            self._format_interaction(interaction)
            for interaction in self.content
        ]
        return "".join(formatted)

    def update(self, interaction: UserInteraction):
        # Keep only the last `window_size` interactions, then add the new one
        self.content = self.content[-self.window_size:] + [interaction]

    def __str__(self):
        return self.render()


# =====================
# Signature and ChainOfThought for Responding with Context
# =====================

class ResponseWithContext(dspy.Signature):
    """Respond to the user's message given some conversation context."""
    context = dspy.InputField(desc="The context of the conversation")
    message = dspy.InputField(desc="The user's message")
    rag_context = dspy.InputField(desc="Relevant data from observing metadata")
    response = dspy.OutputField(desc="A useful response to the user's message within the context of the conversation")

respond_cot = dspy.ChainOfThought(ResponseWithContext)

# =====================
# Retrieval Function (RAG)
# =====================
def getObservingData(dayObs=None):
    """Get the observing metadata for the current or a past day.

    Parameters
    ----------
    dayObs : `int`, optional
        The day for which to get the observing metadata. If not specified,
        the current day is used.

    Returns
    -------
    observingData: `pandas.DataFrame`
        The observing metadata for the specified day.
    """
    currentDayObs = 20240904
    if dayObs is None:
        dayObs = currentDayObs
    isCurrent = (dayObs == currentDayObs)

    site = "local"  # Example site setting
    if site == "local":
        filename = f"/home/c/carlosm/lsst/summit_extras/python/lsst/summit/extras/dayObs_{dayObs}.json"
    elif site in ["rubin-devl"]:
        LOG.warning(
            f"Observing metadata at {site} is manually copied. It may not be updated in real-time."
        )
        filename = f"/sdf/home/m/mfl/u/rubinTvDataProducts/sidecar_metadata/dayObs_{dayObs}.json"
    elif site in ["staff-rsp"]:
        LOG.warning(
            f"Observing metadata at {site} is manually copied. It may not be updated in real-time."
        )
        filename = f"/home/m/mfl/u/rubinTvDataProducts/sidecar_metadata/dayObs_{dayObs}.json"
    else:
        raise RuntimeError(f"Observing metadata not available for site: {site}")

    if not os.path.exists(filename):
        LOG.warning(
            f"Observing metadata file for {'current ' if isCurrent else ''}dayObs "
            f"{dayObs} does not exist at {filename}."
        )
        return pd.DataFrame()

    try:
        table = pd.read_json(filename).T
    except ValueError as e:
        LOG.error(f"Error reading JSON file {filename}: {e}")
        return pd.DataFrame()

    # Remove columns starting with underscore (backend only)
    table = table.drop([col for col in table.columns if col.startswith("_")], axis=1, errors='ignore')
    return table.sort_index()


def retrieve_observing_metadata(query, dayObs=None):
    """Retrieve relevant observing data based on the query."""
    df = getObservingData(dayObs=dayObs)
    if df.empty:
        return "No observing data available."
    
    # Filter or search the dataframe based on the query
    result = df[df.apply(lambda row: query.lower() in row.to_string().lower(), axis=1)]
    if result.empty:
        return "No relevant data found."
    else:
        return result.to_string(index=False)

# =====================
# ProgramOfThought Class (Updated for RAG)
# =====================

class ProgramOfThought(dspy.Module):
    """Class to generate and execute Python code iteratively to solve questions, now with context consideration."""

    def __init__(self, signature, max_iters=3):
        super().__init__()
        self.signature = signature
        self.max_iters = max_iters
        self.interpreter = PythonInterpreter(
            action_space={"print": print},
            import_white_list=["numpy", "astropy", "sympy", "matplotlib"]
        )

    def forward(self, question, variables=None, context_str="", rag_context=""):
        """Generates and executes Python code for a given question."""
        for iteration in range(self.max_iters):
            generated_code = self._generate_code(question, context_str, rag_context)
            print(f"\nIteration {iteration + 1}:")
            print("Generated Code:\n", generated_code)

            try:
                result, _ = CodePrompt(generated_code).execute(
                    self.interpreter,
                    user_variable=variables
                )
                print("Execution Result:\n", result)
                return {"answer": result}
            except Exception as e:
                LOG.error(f"Execution Error: {e}")

        print("\nFailed to generate valid code after several iterations.")
        return {"answer": None}

    def _generate_code(self, question, context_str="", rag_context=""):
        """Generates Python code using the model with RAG data."""
        generate_code_signature = dspy.Signature("question -> generated_code")
        predict = dspy.Predict(generate_code_signature)

        prompt = f"""
You are an AI that writes Python code to solve problems. 
Below is the conversation context and relevant observing metadata:

Context:
{context_str}

Observing Metadata:
{rag_context}

Now, generate Python code to solve the following problem. 
Do not include any explanations, Markdown formatting, or comments. 
Only output the executable Python code.

Question: {question}

Code:
"""
        completion = predict(question=prompt)
        generated_code = completion.generated_code

        # Remove any triple-backtick fences
        cleaned_code = re.sub(r"```[^\n]*\n", "", generated_code)
        cleaned_code = re.sub(r"```", "", cleaned_code)

        return cleaned_code

# =====================
# Main Loop with RAG Integration
# =====================
if __name__ == '__main__':
    context = ConversationContext(window_size=5)
    generate_answer_signature = dspy.Signature("question -> answer")
    pot = ProgramOfThought(signature=generate_answer_signature)

    while True:
        try:
            user_message = input(">>> ")

            if user_message.strip().lower() == ':exit':
                break
            elif user_message.strip().lower() == ':history':
                print(context.render())
                continue

            # Retrieve RAG context from local JSON
            rag_context = retrieve_observing_metadata(user_message)

            # Generate a short text response to the user
            assistant_response = respond_cot(
                context=context.render(),
                message=user_message,
                rag_context=rag_context
            ).response

            print(f'\n<<< {assistant_response}\n')

            # Update conversation memory
            interaction = UserInteraction(message=user_message, response=assistant_response)
            context.update(interaction)

            # If the user wants a calculation/solution, invoke ProgramOfThought
            if "calculate" in user_message.lower() or "solve" in user_message.lower():
                variables = {"gravity": 9.81}
                result = pot(
                    question=user_message,
                    variables=variables,
                    context_str=context.render(),
                    rag_context=rag_context
                )
                print("ProgramOfThought Answer:", result['answer'])

        except KeyboardInterrupt:
            break

    print('\nBye!')